## Framework

- Data cleaning and formatting 
- Exploratory data analysis
- Feature engineering and selection
- Compare several machine learning models on a performance metric
- Perform hyper-parameter tuning on the best model 
- Evaluate the best model on the testing set
- Interpret the model results
- Draw conclusions and document work

In [1]:
#load packages

#lm pacakges
from sklearn import tree 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.preprocessing import normalize, scale, Normalizer, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.dummy import DummyClassifier

#other 
import numpy as np
import pandas as pd
import pickle 
import graphviz
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling
pd.set_option("display.max_colwidth", 200)

import altair as alt
import time

import autotime

//anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

In [3]:
df1 = pd.read_csv('data/01_oscar_data_2019.csv')


In [4]:
#df1.profile_report(style={'full_width':True})

- remove all the categorical for wins and nominations so model only is concern with # of nominations and wins 
    - might have to return to this later

In [5]:
df1.head(1)

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,gross,release_date,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Oscar_nominated_categories,Golden_Globes_won,Golden_Globes_won_categories,Golden_Globes_nominated,Golden_Globes_nominated_categories,BAFTA_won,BAFTA_won_categories,BAFTA_nominated,BAFTA_nominated_categories,Screen_Actors_Guild_won,Screen_Actors_Guild_won_categories,Screen_Actors_Guild_nominated,Screen_Actors_Guild_nominated_categories,Critics_Choice_won,Critics_Choice_won_categories,Critics_Choice_nominated,Critics_Choice_nominated_categories,Directors_Guild_won,Directors_Guild_won_categories,Directors_Guild_nominated,Directors_Guild_nominated_categories,Producers_Guild_won,Producers_Guild_won_categories,Producers_Guild_nominated,Producers_Guild_nominated_categories,Art_Directors_Guild_won,Art_Directors_Guild_won_categories,Art_Directors_Guild_nominated,Art_Directors_Guild_nominated_categories,Writers_Guild_won,Writers_Guild_won_categories,Writers_Guild_nominated,Writers_Guild_nominated_categories,Costume_Designers_Guild_won,Costume_Designers_Guild_won_categories,Costume_Designers_Guild_nominated,Costume_Designers_Guild_nominated_categories,Online_Film_Television_Association_won,Online_Film_Television_Association_won_categories,Online_Film_Television_Association_nominated,Online_Film_Television_Association_nominated_categories,Online_Film_Critics_Society_won,Online_Film_Critics_Society_won_categories,Online_Film_Critics_Society_nominated,Online_Film_Critics_Society_nominated_categories,People_Choice_won,People_Choice_won_categories,People_Choice_nominated,People_Choice_nominated_categories,London_Critics_Circle_Film_won,London_Critics_Circle_Film_won_categories,London_Critics_Circle_Film_nominated,London_Critics_Circle_Film_nominated_categories,American_Cinema_Editors_won,American_Cinema_Editors_won_categories,American_Cinema_Editors_nominated,American_Cinema_Editors_nominated_categories,Hollywood_Film_won,Hollywood_Film_won_categories,Hollywood_Film_nominated,Hollywood_Film_nominated_categories,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_won_categories,Austin_Film_Critics_Association_nominated,Austin_Film_Critics_Association_nominated_categories,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_won_categories,Denver_Film_Critics_Society_nominated,Denver_Film_Critics_Society_nominated_categories,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_won_categories,Boston_Society_of_Film_Critics_nominated,Boston_Society_of_Film_Critics_nominated_categories,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_won_categories,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
0,2001,Kate & Leopold,tt0035423,PG-13,118,Comedy|Fantasy|Romance,6.4,44.0,An English Duke from 1876 is inadvertedly dragged to modern day New York where he falls for a plucky advertising executive.,66660,47100000.0,2001-12-25,318.0,125.0,2363.0,1,4,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1,"Best Music, Original Song",1,Best Original Song - Motion Picture,2,Best Original Song - Motion Picture|Best Performance by an Actor in a Motion Picture - Comedy

# Data cleaning and formatting and Exploratory data analysis

**BAFTA Have not happened yet drop the awards**

In [6]:
#added back in golden globes for test
df1_temp = df1.drop(['movie_id', 'release_date', 'synopsis', 'release_date.year','release_date.month', 'release_date.day-of-month', 'release_date.day-of-week', 'American_Cinema_Editors_nominated_categories', 'Art_Directors_Guild_nominated_categories', 'Austin_Film_Critics_Association_nominated_categories', 'BAFTA_nominated_categories', 'Boston_Society_of_Film_Critics_nominated_categories', 'Costume_Designers_Guild_nominated_categories', 'Costume_Designers_Guild_won_categories', 'Critics_Choice_nominated_categories', 'Denver_Film_Critics_Society_nominated_categories', 'Directors_Guild_nominated_categories','Golden_Globes_nominated_categories','Hollywood_Film_nominated_categories', 'London_Critics_Circle_Film_nominated_categories', 'Los_Angeles_Film_Critics_Association_nominated_categories', 'New_York_Film_Critics_Circle_nominated_categories', 'Online_Film_Critics_Society_nominated_categories', 'Online_Film_Television_Association_nominated_categories', 'People_Choice_nominated_categories',  'Producers_Guild_nominated_categories', 'Screen_Actors_Guild_nominated_categories', 'Writers_Guild_nominated_categories', 'American_Cinema_Editors_won_categories', 'London_Critics_Circle_Film_won_categories', 'BAFTA_won_categories', 'BAFTA_won_categories', 'BAFTA_nominated_categories', 'BAFTA_won'], axis = 1)

In [7]:
df1.head(0)

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,gross,release_date,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Oscar_nominated_categories,Golden_Globes_won,Golden_Globes_won_categories,Golden_Globes_nominated,Golden_Globes_nominated_categories,BAFTA_won,BAFTA_won_categories,BAFTA_nominated,BAFTA_nominated_categories,Screen_Actors_Guild_won,Screen_Actors_Guild_won_categories,Screen_Actors_Guild_nominated,Screen_Actors_Guild_nominated_categories,Critics_Choice_won,Critics_Choice_won_categories,Critics_Choice_nominated,Critics_Choice_nominated_categories,Directors_Guild_won,Directors_Guild_won_categories,Directors_Guild_nominated,Directors_Guild_nominated_categories,Producers_Guild_won,Producers_Guild_won_categories,Producers_Guild_nominated,Producers_Guild_nominated_categories,Art_Directors_Guild_won,Art_Directors_Guild_won_categories,Art_Directors_Guild_nominated,Art_Directors_Guild_nominated_categories,Writers_Guild_won,Writers_Guild_won_categories,Writers_Guild_nominated,Writers_Guild_nominated_categories,Costume_Designers_Guild_won,Costume_Designers_Guild_won_categories,Costume_Designers_Guild_nominated,Costume_Designers_Guild_nominated_categories,Online_Film_Television_Association_won,Online_Film_Television_Association_won_categories,Online_Film_Television_Association_nominated,Online_Film_Television_Association_nominated_categories,Online_Film_Critics_Society_won,Online_Film_Critics_Society_won_categories,Online_Film_Critics_Society_nominated,Online_Film_Critics_Society_nominated_categories,People_Choice_won,People_Choice_won_categories,People_Choice_nominated,People_Choice_nominated_categories,London_Critics_Circle_Film_won,London_Critics_Circle_Film_won_categories,London_Critics_Circle_Film_nominated,London_Critics_Circle_Film_nominated_categories,American_Cinema_Editors_won,American_Cinema_Editors_won_categories,American_Cinema_Editors_nominated,American_Cinema_Editors_nominated_categories,Hollywood_Film_won,Hollywood_Film_won_categories,Hollywood_Film_nominated,Hollywood_Film_nominated_categories,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_won_categories,Austin_Film_Critics_Association_nominated,Austin_Film_Critics_Association_nominated_categories,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_won_categories,Denver_Film_Critics_Society_nominated,Denver_Film_Critics_Society_nominated_categories,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_won_categories,Boston_Society_of_Film_Critics_nominated,Boston_Society_of_Film_Critics_nominated_categories,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_won_categories,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week


In [8]:
#df1_temp.profile_report(style={'full_width':True})

- Remove Hollywood_Film_won and Hollywood_Film_nominated do to high correlation 
- Can keep Online_Film_Television_Association_nominated and awards_nominations because it does not look like they are related
- give gross and popularity missing values the mediumn value
- remove the missing metascore, certificate, and release_date.month values 
- Deal with the categorical nature of genre using MultiLabelBinarizer

In [9]:
df1_temp = df1_temp.drop(['Hollywood_Film_won', 'Hollywood_Film_nominated'], axis = 1)

In [10]:
df1_temp = df1_temp.dropna(subset=['metascore', 'certificate'])

In [11]:
df1_temp['gross'].fillna((df1_temp['gross'].median()), inplace=True)
df1_temp['popularity'].fillna((df1_temp['popularity'].median()), inplace=True)

### Factor features 

In [12]:
#this one was done before the other factor features and will be left alone. 
df1_temp['genre'] = [x.split('|') for x in df1_temp['genre']]

In [13]:
mlb = MultiLabelBinarizer()
df1_temp = df1_temp.join(pd.DataFrame(mlb.fit_transform(df1_temp.pop('genre')),
                          columns=mlb.classes_,
                          index=df1_temp.index))

In [14]:
factor_features = ['Art_Directors_Guild_won_categories', 'Austin_Film_Critics_Association_won_categories', 'Boston_Society_of_Film_Critics_won_categories', 'Critics_Choice_won_categories', 'Denver_Film_Critics_Society_won_categories', 'Directors_Guild_won_categories', 'Golden_Globes_won_categories', 'Hollywood_Film_won_categories', 'Los_Angeles_Film_Critics_Association_won_categories', 'New_York_Film_Critics_Circle_won_categories', 'Online_Film_Critics_Society_won_categories', 'Online_Film_Television_Association_won_categories', 'Oscar_nominated_categories','People_Choice_won_categories','Screen_Actors_Guild_won_categories','Writers_Guild_won_categories', 'Producers_Guild_won_categories']
awards = ['Art Directors Guild', 'Austin Film Critics Association',  'Boston Society of Film Critics', 'Critics Choice', 'Denver Film Critics Society', 'Directors Guild', 'Golden Globes', 'Hollywood Film', 'Los Angeles Film Critics Association', 'New York Film Critics Circle', 'Online Film Critics Society', 'Online Film Television Association', 'Oscar nominated','People Choice','Screen Actors Guild','Writers Guild', 'Producers Guild']
#'American_Cinema_Editors_won_categories', 'London_Critics_Circle_Film_won_categories',
#'American Cinema Editors', 'London Critics Circle Film',

In [15]:
mlb = MultiLabelBinarizer()

for feature, award in zip(factor_features, awards):
    df1_temp[feature] = df1_temp[feature].astype(str)
    df1_temp[feature] = [x.replace('Best', f'{award} Best') for x in df1_temp[feature]]
    df1_temp[feature] = [x.split('|') for x in df1_temp[feature]]
    df1_temp = df1_temp.join(pd.DataFrame(mlb.fit_transform(df1_temp.pop(feature)),
                             columns=mlb.classes_,
                             index=df1_temp.index))
    df1_temp = df1_temp.drop(['nan'], axis = 1)
    df1_temp[''] = 1
    df1_temp = df1_temp.drop([''], axis = 1)
    df1_temp['Animation'] = 1
    df1_temp = df1_temp.drop(['Animation'], axis = 1)
    print(award)
    

Art Directors Guild
Austin Film Critics Association
Boston Society of Film Critics
Critics Choice
Denver Film Critics Society
Directors Guild
Golden Globes
Hollywood Film
Los Angeles Film Critics Association
New York Film Critics Circle
Online Film Critics Society
Online Film Television Association
Oscar nominated
People Choice
Screen Actors Guild
Writers Guild
Producers Guild


In [16]:
df1_temp.head(0)

,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Histor,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Ensemble,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Boston Society of Film Critics Best Actor,Boston Society of Film Critics Best Actress,Boston Society of Film Critics Best Animated Film,Boston Society of Film Critics Best Cinematography,Boston Society of Film Critics Best Director,Boston Society of Film Critics Best Ensemble Cast,Boston Society of Film Critics Best Film,Boston Society of Film Critics Best Film Editing,Boston Society of Film Critics Best Foreign Language Film,Boston Society of Film Critics Best New Filmmaker,Boston Society of Film Critics Best Original Score,Boston Society of Film Critics Best Picture,Boston Society of Film Critics Best Screenplay,Boston Society of Film Critics Best Supporting Actor,Boston Society of Film Critics Best Supporting Actress,Boston Society of Film Critics Best Use of Music in a Film,Breakthrough Artist,Critics Choice Best Acting Ensemble,Critics Choice Best Action Movie,Critics Choice Best Actor,Critics Choice Best Actor in a Comedy,Critics Choice Best Actor in a Comedy

In [17]:
# df1_temp['Golden_Globes_won_categories'] = [x.split('|') for x in df1_temp['Golden_Globes_won_categories']]


In [18]:
# df1_temp.profile_report(style={'full_width':True})

- Fix the histor category history 

In [19]:
df1_temp = df1_temp.reset_index()

In [20]:
df1_temp.query("Histor == 1")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Histor,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Ensemble,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Boston Society of Film Critics Best Actor,Boston Society of Film Critics Best Actress,Boston Society of Film Critics Best Animated Film,Boston Society of Film Critics Best Cinematography,Boston Society of Film Critics Best Director,Boston Society of Film Critics Best Ensemble Cast,Boston Society of Film Critics Best Film,Boston Society of Film Critics Best Film Editing,Boston Society of Film Critics Best Foreign Language Film,Boston Society of Film Critics Best New Filmmaker,Boston Society of Film Critics Best Original Score,Boston Society of Film Critics Best Picture,Boston Society of Film Critics Best Screenplay,Boston Society of Film Critics Best Supporting Actor,Boston Society of Film Critics Best Supporting Actress,Boston Society of Film Critics Best Use of Music in a Film,Breakthrough Artist,Critics Choice Best Acting Ensemble,Critics Choice Best Action Movie,Critics Choice Best Actor,Critics Choice Best Actor in a Comedy,Critics Choice Best Actor in a 

In [21]:
df1_temp.iloc[682]["History"] = 1

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
df1_temp = df1_temp.drop(["Histor", "index"], axis=1) 

- fix the certified column to move tv-ma to r and unrated to not rated

In [23]:
df1_temp = df1_temp.reset_index()

In [24]:
df1_temp.query("certificate == 'TV-MA'")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Ensemble,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Boston Society of Film Critics Best Actor,Boston Society of Film Critics Best Actress,Boston Society of Film Critics Best Animated Film,Boston Society of Film Critics Best Cinematography,Boston Society of Film Critics Best Director,Boston Society of Film Critics Best Ensemble Cast,Boston Society of Film Critics Best Film,Boston Society of Film Critics Best Film Editing,Boston Society of Film Critics Best Foreign Language Film,Boston Society of Film Critics Best New Filmmaker,Boston Society of Film Critics Best Original Score,Boston Society of Film Critics Best Picture,Boston Society of Film Critics Best Screenplay,Boston Society of Film Critics Best Supporting Actor,Boston Society of Film Critics Best Supporting Actress,Boston Society of Film Critics Best Use of Music in a Film,Breakthrough Artist,Critics Choice Best Acting Ensemble,Critics Choice Best Action Movie,Critics Choice Best Actor,Critics Choice Best Actor in a Comedy,Critics Choice Best Actor in a Comedy 

In [25]:
df1_temp.loc[df1_temp['certificate'].str.contains('TV-MA', case=False), 'certificate'] = 'R'

In [26]:
df1_temp.iloc[1159]['certificate'] 

'R'

In [27]:
df1_temp.loc[df1_temp['certificate'].str.contains('Unrated', case=False), 'certificate'] = 'Not Rated'

In [28]:
df1_temp.query("certificate == 'Unrated'")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Ensemble,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Boston Society of Film Critics Best Actor,Boston Society of Film Critics Best Actress,Boston Society of Film Critics Best Animated Film,Boston Society of Film Critics Best Cinematography,Boston Society of Film Critics Best Director,Boston Society of Film Critics Best Ensemble Cast,Boston Society of Film Critics Best Film,Boston Society of Film Critics Best Film Editing,Boston Society of Film Critics Best Foreign Language Film,Boston Society of Film Critics Best New Filmmaker,Boston Society of Film Critics Best Original Score,Boston Society of Film Critics Best Picture,Boston Society of Film Critics Best Screenplay,Boston Society of Film Critics Best Supporting Actor,Boston Society of Film Critics Best Supporting Actress,Boston Society of Film Critics Best Use of Music in a Film,Breakthrough Artist,Critics Choice Best Acting Ensemble,Critics Choice Best Action Movie,Critics Choice Best Actor,Critics Choice Best Actor in a Comedy,Critics Choice Best Actor in a Comedy 

In [29]:
df1_temp = df1_temp.dropna()

In [30]:
df_clean = df1_temp

In [31]:
df_clean

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Ensemble,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Boston Society of Film Critics Best Actor,Boston Society of Film Critics Best Actress,Boston Society of Film Critics Best Animated Film,Boston Society of Film Critics Best Cinematography,Boston Society of Film Critics Best Director,Boston Society of Film Critics Best Ensemble Cast,Boston Society of Film Critics Best Film,Boston Society of Film Critics Best Film Editing,Boston Society of Film Critics Best Foreign Language Film,Boston Society of Film Critics Best New Filmmaker,Boston Society of Film Critics Best Original Score,Boston Society of Film Critics Best Picture,Boston Society of Film Critics Best Screenplay,Boston Society of Film Critics Best Supporting Actor,Boston Society of Film Critics Best Supporting Actress,Boston Society of Film Critics Best Use of Music in a Film,Breakthrough Artist,Critics Choice Best Acting Ensemble,Critics Choice Best Action Movie,Critics Choice Best Actor,Critics Choice Best Actor in a Comedy,Critics Choice Best Actor in a Comedy 

In [32]:
# #structure features 
# df_clean = df_clean.drop(['index'], axis = 1)
# df_clean.head(1)

### Pull the winners of 2018 for back testing 

In [33]:
best_picture_2019_identifiable = df_clean.query("Oscar_Best_Picture_nominated == 'Yes' & year == 2019")
best_director_2019_identifiable = df_clean.query("Oscar_Best_Director_nominated == 'Yes' & year == 2019")
best_actor_2019_identifiable = df_clean.query("Oscar_Best_Actor_nominated == 'Yes' & year == 2019")
best_actress_2019_identifiable = df_clean.query("Oscar_Best_Actress_nominated == 'Yes' & year == 2019")
best_supporting_actor_2019_identifiable = df_clean.query("Oscar_Best_Supporting_Actor_nominated == 'Yes' & year == 2019")
best_supporting_actress_2019_identifiable = df_clean.query("Oscar_Best_Supporting_Actress_nominated == 'Yes' & year == 2019")
best_oriscreen_2019_identifiable = df_clean.query("Oscar_Best_OriScreen_nominated == 'Yes' & year == 2019")
best_adascreen_2019_identifiable = df_clean.query("Oscar_Best_AdaScreen_nominated == 'Yes' & year == 2019")


In [34]:
best_picture_2019_identifiable

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Ensemble,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Boston Society of Film Critics Best Actor,Boston Society of Film Critics Best Actress,Boston Society of Film Critics Best Animated Film,Boston Society of Film Critics Best Cinematography,Boston Society of Film Critics Best Director,Boston Society of Film Critics Best Ensemble Cast,Boston Society of Film Critics Best Film,Boston Society of Film Critics Best Film Editing,Boston Society of Film Critics Best Foreign Language Film,Boston Society of Film Critics Best New Filmmaker,Boston Society of Film Critics Best Original Score,Boston Society of Film Critics Best Picture,Boston Society of Film Critics Best Screenplay,Boston Society of Film Critics Best Supporting Actor,Boston Society of Film Critics Best Supporting Actress,Boston Society of Film Critics Best Use of Music in a Film,Breakthrough Artist,Critics Choice Best Acting Ensemble,Critics Choice Best Action Movie,Critics Choice Best Actor,Critics Choice Best Actor in a Comedy,Critics Choice Best Actor in a Comedy 

In [35]:
t = pd.concat([best_actor_2019_identifiable, best_actress_2019_identifiable, best_adascreen_2019_identifiable, best_director_2019_identifiable, best_oriscreen_2019_identifiable, best_picture_2019_identifiable, best_supporting_actor_2019_identifiable, best_supporting_actress_2019_identifiable], axis=0)
best_back_test_2019_identifiable = t.drop_duplicates(keep='last')

df_ml = df_clean.drop(list(best_back_test_2019_identifiable.index))


In [36]:
#final drop of identifable information
df_ml = df_ml.drop(['year', 'movie', 'index'], axis = 1)
best_back_test_2019_unidentifiable = best_back_test_2019_identifiable.drop(['year', 'movie', 'index'], axis = 1)

In [37]:
df_ml

,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Animated Film,Contemporary Film,Fantasy Film,Period Film,Period or Fantasy Film,"Variety or Awards Show, Music Special or Documentary",Austin Film Award,Austin Film Critics Association Best Actor,Austin Film Critics Association Best Actress,Austin Film Critics Association Best Adapted Screenplay,Austin Film Critics Association Best Animated Feature,Austin Film Critics Association Best Animated Film,Austin Film Critics Association Best Cinematography,Austin Film Critics Association Best Director,Austin Film Critics Association Best Ensemble,Austin Film Critics Association Best Film,Austin Film Critics Association Best First Film,Austin Film Critics Association Best Foreign Film,Austin Film Critics Association Best Foreign Language Film,Austin Film Critics Association Best Movie of the Decade,Austin Film Critics Association Best Original Score,Austin Film Critics Association Best Original Screenplay,Austin Film Critics Association Best Score,Austin Film Critics Association Best Supporting Actor,Austin Film Critics Association Best Supporting Actress,Breakthrough Artist Award,Most Promising Filmmaker,Special Honorary Award,Boston Society of Film Critics Best Actor,Boston Society of Film Critics Best Actress,Boston Society of Film Critics Best Animated Film,Boston Society of Film Critics Best Cinematography,Boston Society of Film Critics Best Director,Boston Society of Film Critics Best Ensemble Cast,Boston Society of Film Critics Best Film,Boston Society of Film Critics Best Film Editing,Boston Society of Film Critics Best Foreign Language Film,Boston Society of Film Critics Best New Filmmaker,Boston Society of Film Critics Best Original Score,Boston Society of Film Critics Best Picture,Boston Society of Film Critics Best Screenplay,Boston Society of Film Critics Best Supporting Actor,Boston Society of Film Critics Best Supporting Actress,Boston Society of Film Critics Best Use of Music in a Film,Breakthrough Artist,Critics Choice Best Acting Ensemble,Critics Choice Best Action Movie,Critics Choice Best Actor,Critics Choice Best Actor in a Comedy,Critics Choice Best Actor in a Comedy Movie,Critics Cho

### Create the X and Ys

In [38]:
X = df_ml.drop(['Oscar_Best_Actor_won', 'Oscar_Best_Actress_won', 'Oscar_Best_AdaScreen_won', 'Oscar_Best_Director_won', 'Oscar_Best_OriScreen_won', 'Oscar_Best_Picture_won', "Oscar_Best_Supporting_Actor_won", "Oscar_Best_Supporting_Actress_won"], axis = 1)
y_best_actor = df_ml['Oscar_Best_Actor_won']
y_best_actress = df_ml['Oscar_Best_Actress_won']
y_best_ada_screen = df_ml['Oscar_Best_AdaScreen_won']
y_best_director = df_ml['Oscar_Best_Director_won']
y_best_ori_screen = df_ml['Oscar_Best_OriScreen_won']
y_best_picture = df_ml['Oscar_Best_Picture_won']
y_best_supporting_actor = df_ml['Oscar_Best_Supporting_Actor_won']
y_best_supporting_actress = df_ml['Oscar_Best_Supporting_Actress_won']

### Split X and Y 

In [39]:
#best actor
X_train_best_actor, X_test_best_actor, y_train_best_actor, y_test_best_actor = train_test_split(X,
                                                                          y_best_actor,
                                                                          test_size=0.2)
#best actoress 
X_train_best_actress, X_test_best_actress, y_train_best_actress, y_test_best_actress = train_test_split(X,
                                                                          y_best_actress,
                                                                          test_size=0.2)

#best adaptive screen play 
X_train_best_ada_screen, X_test_best_ada_screen, y_train_best_ada_screen, y_test_best_ada_screen = train_test_split(X,
                                                                          y_best_ada_screen,
                                                                          test_size=0.2)
#best director
X_train_best_director, X_test_best_director, y_train_best_director, y_test_best_director = train_test_split(X,
                                                                          y_best_director,
                                                                          test_size=0.2)

#best orginal screen play 
X_train_best_ori_screen, X_test_best_ori_screen, y_train_best_ori_screen, y_test_best_ori_screen = train_test_split(X,
                                                                          y_best_ori_screen,
                                                                          test_size=0.2)

#best picture 
X_train_best_picture, X_test_best_picture, y_train_best_picture, y_test_best_picture = train_test_split(X,
                                                                          y_best_picture,
                                                                          test_size=0.2)

#best supporting actor 
X_train_best_supporting_actor, X_test_best_supporting_actor, y_train_best_supporting_actor, y_test_best_supporting_actor = train_test_split(X,
                                                                          y_best_supporting_actor,
                                                                          test_size=0.2)

#best suporting actress 
X_train_best_supporting_actress, X_test_best_supporting_actress, y_train_best_supporting_actress, y_test_best_supporting_actress = train_test_split(X,
                                                                          y_best_supporting_actress,
                                                                          test_size=0.2)


In [40]:
#split 
numeric_features = ['duration', 'rate', 'metascore', 'gross', 'user_reviews', 'critic_reviews', 'popularity']
    
    
categorical_features = ['certificate', 'Oscar_Best_Picture_nominated', 'Oscar_Best_Director_nominated', 'Oscar_Best_Actor_nominated', 'Oscar_Best_Actress_nominated', 'Oscar_Best_Supporting_Actress_nominated', 'Oscar_Best_OriScreen_nominated']

### Processing Pipeline 

In [41]:
# transform the data to standardize the values in the data 
preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), numeric_features),
        ('ohe', OneHotEncoder(drop="first"), categorical_features)])


In [42]:
def get_scores(model, 
                X_train, y_train,
                X_test, y_test, 
                show = True
               ):
    """
    Returns train and validation error given a model
    train and validation X and y portions
    Parameters
    ----------
    model: sklearn classifier model
        The sklearn model
    X_train: numpy.ndarray        
        The X part of the train set
    y_train: numpy.ndarray
        The y part of the train set    
    X_valid: numpy.ndarray        
        The X part of the validation set
    y_valid: numpy.ndarray
        The y part of the validation set    
    Returns
    -------
        train_err: float
        test_err: float
            
    """ 
    
    if show: 
        print("Training error:   %.2f" % (1-model.score(X_train, y_train)))
        print("Validation error: %.2f" % (1-model.score(X_test, y_test)))
        print('\n')
    return (1-model.score(X_train, y_train)), (1-model.score(X_test, y_test))

In [43]:
def diff_class_ml(X_train, X_test, y_train, y_test):
    """
    Returns train error, validation error and time given an ensemble of models 
    Parameters
    ----------
    X_train: numpy.ndarray        
        The X part of the train set
    y_train: numpy.ndarray
        The y part of the train set    
    X_valid: numpy.ndarray        
        The X part of the validation set
    y_valid: numpy.ndarray
        The y part of the validation set    
    Returns
    -------
        pd.DataFrame
            
    """ 
    # Lets create an empty dictionary to store all the results
    results_dict = {}
    
    models = {
          'dummy': DummyClassifier(), 
          'decision tree': DecisionTreeClassifier(),
          'kNN': KNeighborsClassifier(),
          'logistic regression': LogisticRegression(),
          'random forest' : RandomForestClassifier(), 
          'xgboost' : XGBClassifier(),
          'lgbm': LGBMClassifier()
         }

    for model_name, model in models.items():
        t = time.time()
        #print(model_name, ":")    
        clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', model)])
        clf.fit(X_train, y_train);
        tr_err, valid_err = get_scores(clf, X_train, y_train, 
                                       X_test, y_test, show = False)
        elapsed_time = time.time() - t
        results_dict[model_name] = [round(tr_err,3), round(valid_err,3), round(elapsed_time,4)]
        #print("Elapsed time: %.1f s" % elapsed_time)
    
    results_df = pd.DataFrame(results_dict).T
    results_df.columns = ["Train error", "Validation error", "Time in seconds"]
    return results_df

In [44]:
def oscar_predict(X_train, y_train, dataframe, model):
    """
    Shows Classifier and Probability for the Oscar nomination (or other movie)
    
    Parameters
    ----------
    X_train: numpy.ndarray 
        The X training set of the selected catagory 
    y_train: numpy.ndarray
        The y training set the selected catagory 
    dataframe: numpy.ndarray
        The movies that need to be predicted if they won their category 
    model: sklearn classifier model
        The sklearn model such as LogisticRegression()
    
    Returns
    -------
        pd.DataFrame
    
    """
    # Lets create an empty dictionary to store all the results
    results_dict = {}
    
    movies = list(dataframe.index)
    movies_names = list(dataframe["movie"])
    dataframe_X = dataframe.drop(['year', 'movie', 'index', 'Oscar_Best_Actor_won', 'Oscar_Best_Actress_won', 'Oscar_Best_AdaScreen_won', 'Oscar_Best_Director_won', 'Oscar_Best_OriScreen_won', 'Oscar_Best_Picture_won', "Oscar_Best_Supporting_Actor_won", "Oscar_Best_Supporting_Actress_won"], axis = 1)

    clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', model)])
    model = clf.fit(X_train, y_train);
        
    for movie, movie_name in zip(movies, movies_names):
        movie_predict = dataframe_X.loc[[movie]]
        classifier = model.predict(movie_predict)
        classifier = classifier[0]
        prob = model.predict_proba(movie_predict)
        prob = prob[0][1]
        results_dict[movie_name] = [classifier, round(prob,8)]
    
    results_df = pd.DataFrame(results_dict).T
    results_df.columns = ["Classifier", "Probability of it Winning"]
    return results_df

In [45]:
def normalized_winner(predict_df):
    """
    Normalized the chance of a movie winning an oscar
    
    Parameters
    ----------
    predict_df: pd.DataFram from oscar_predict()
        
    Returns
    -------
        pd.DataFrame
    
    """
    predict_df = predict_df.drop(["Classifier"], axis = 1)
    predict_df["Chance of Winning"] = predict_df["Probability of it Winning"]/(predict_df["Probability of it Winning"].sum())
    predict_df = predict_df.drop(["Probability of it Winning"], axis = 1)
    predict_df ["Chance of Winning"] = pd.Series(["{0:.2f}%".format(val * 100) for val in predict_df ["Chance of Winning"]], index = predict_df.index)
    return predict_df

### Base Model Best Picture

In [46]:
diff_class_ml(X_train_best_picture, X_test_best_picture, y_train_best_picture, y_test_best_picture)

,Train error,Validation error,Time in seconds
dummy,0.028,0.013,0.0198
decision tree,0.000,0.025,0.0167
kNN,0.014,0.013,0.0580
logistic regression,0.017,0.013,0.0269
random forest,0.000,0.013,0.1670
xgboost,0.000,0.017,0.1004
lgbm,0.000,0.013,0.0865


In [47]:
predict_picture_2019 = oscar_predict(X_train_best_picture, y_train_best_picture, best_picture_2019_identifiable, LogisticRegression())
predict_picture_2019

,Classifier,Probability of it Winning
Jojo Rabbit,No,0.0254467
1917,No,0.174032
Parasite,No,0.365329
Little Woman,No,0.0287527
Ford v Ferrari,No,0.022694
Marriage Story,No,0.10866
Joker,Yes,0.973474
The Irishman,No,0.281067
Once Upon a Time... in Hollywood,Yes,0.799245


In [48]:
normalized_winner(predict_picture_2019)

,Chance of Winning
Jojo Rabbit,0.92%
1917,6.26%
Parasite,13.15%
Little Woman,1.03%
Ford v Ferrari,0.82%
Marriage Story,3.91%
Joker,35.03%
The Irishman,10.12%
Once Upon a Time... in Hollywood,28.76%


### Base Model Best Actor

In [49]:
diff_class_ml(X_train_best_actor, X_test_best_actor, y_train_best_actor, y_test_best_actor)

,Train error,Validation error,Time in seconds
dummy,0.026,0.029,0.0208
decision tree,0.000,0.029,0.0205
kNN,0.014,0.025,0.0787
logistic regression,0.012,0.025,0.0317
random forest,0.000,0.025,0.1507
xgboost,0.002,0.025,0.1023
lgbm,0.000,0.021,0.0855


In [70]:
predict_actor_2019 = oscar_predict(X_train_best_actor, y_train_best_actor, best_actor_2019_identifiable, LogisticRegression())
predict_actor_2019

,Classifier,Probability of it Winning
The Two Popes,No,0.0239109
Pain and Glory,No,0.0304507
Marriage Story,No,0.13524
Joker,No,0.00490251
The Irishman,No,0.22514
Once Upon a Time... in Hollywood,No,0.0898148


In [71]:
normalized_winner(predict_actor_2019)

,Chance of Winning
The Two Popes,4.69%
Pain and Glory,5.98%
Marriage Story,26.55%
Joker,0.96%
The Irishman,44.19%
Once Upon a Time... in Hollywood,17.63%


### Base Model Best Actress

In [52]:
diff_class_ml(X_train_best_actress, X_test_best_actress, y_train_best_actress, y_test_best_actress)

,Train error,Validation error,Time in seconds
dummy,0.031,0.029,0.0171
decision tree,0.000,0.025,0.0178
kNN,0.014,0.013,0.0602
logistic regression,0.012,0.013,0.0250
random forest,0.000,0.013,0.1574
xgboost,0.000,0.017,0.1067
lgbm,0.000,0.013,0.0885


In [53]:
predict_actress_2019 = oscar_predict(X_train_best_actress, y_train_best_actress, best_actress_2019_identifiable, LGBMClassifier())
predict_actress_2019

,Classifier,Probability of it Winning
Judy,No,0.00027379
Bombshell,No,0.00028675
Harriet,No,0.00028757
Little Woman,No,0.152494
Marriage Story,No,0.00426014


In [54]:
normalized_winner(predict_actress_2019)

,Chance of Winning
Judy,0.17%
Bombshell,0.18%
Harriet,0.18%
Little Woman,96.76%
Marriage Story,2.70%


### Base Model Best Adapted Screenplay 

In [55]:
diff_class_ml(X_train_best_ada_screen, X_test_best_ada_screen, y_train_best_ada_screen, y_test_best_ada_screen)

,Train error,Validation error,Time in seconds
dummy,0.027,0.038,0.0171
decision tree,0.000,0.029,0.0186
kNN,0.014,0.021,0.0626
logistic regression,0.011,0.025,0.0265
random forest,0.000,0.021,0.1542
xgboost,0.000,0.013,0.1021
lgbm,0.000,0.025,0.0839


In [72]:
predict_ada_screen_2019 = oscar_predict(X_train_best_ada_screen, y_train_best_ada_screen, best_adascreen_2019_identifiable, LogisticRegression())
predict_ada_screen_2019

,Classifier,Probability of it Winning
The Two Popes,No,0.00088108
Jojo Rabbit,No,0.00169326
Little Woman,No,0.0264509
Joker,No,0.02966
The Irishman,Yes,0.631194


In [73]:
normalized_winner(predict_ada_screen_2019)

,Chance of Winning
The Two Popes,0.13%
Jojo Rabbit,0.25%
Little Woman,3.83%
Joker,4.30%
The Irishman,91.49%


### Base Model Best Director

In [58]:
diff_class_ml(X_train_best_director, X_test_best_director, y_train_best_director, y_test_best_director)

,Train error,Validation error,Time in seconds
dummy,0.026,0.025,0.0169
decision tree,0.000,0.013,0.0181
kNN,0.015,0.017,0.0627
logistic regression,0.014,0.017,0.0258
random forest,0.000,0.017,0.1538
xgboost,0.000,0.017,0.1002
lgbm,0.000,0.021,0.0787


In [59]:
predict_director_2019 = oscar_predict(X_train_best_director, y_train_best_director, best_director_2019_identifiable, LGBMClassifier())
predict_director_2019

,Classifier,Probability of it Winning
1917,No,0.00223883
Parasite,Yes,0.673034
Joker,No,0.0247105
The Irishman,Yes,0.929925
Once Upon a Time... in Hollywood,No,0.19447


In [60]:
normalized_winner(predict_director_2019)

,Chance of Winning
1917,0.12%
Parasite,36.89%
Joker,1.35%
The Irishman,50.97%
Once Upon a Time... in Hollywood,10.66%


### Base Model Best Original Screenplay

In [61]:
diff_class_ml(X_train_best_ori_screen, X_test_best_ori_screen, y_train_best_ori_screen, y_test_best_ori_screen)

,Train error,Validation error,Time in seconds
dummy,0.020,0.025,0.0174
decision tree,0.000,0.021,0.0181
kNN,0.012,0.017,0.0610
logistic regression,0.011,0.021,0.0246
random forest,0.000,0.017,0.1540
xgboost,0.000,0.021,0.0995
lgbm,0.000,0.013,0.0805


In [62]:
predict_ori_screen_2019 = oscar_predict(X_train_best_ori_screen, y_train_best_ori_screen, best_oriscreen_2019_identifiable, LGBMClassifier())
predict_ori_screen_2019

,Classifier,Probability of it Winning
Knives Out,No,0.00526421
1917,No,0.00089908
Parasite,Yes,0.885922
Marriage Story,No,0.0125908
Once Upon a Time... in Hollywood,No,0.0556337


In [63]:
normalized_winner(predict_ori_screen_2019)

,Chance of Winning
Knives Out,0.55%
1917,0.09%
Parasite,92.25%
Marriage Story,1.31%
Once Upon a Time... in Hollywood,5.79%


### Base Model Best Supporting Actor

In [64]:
diff_class_ml(X_train_best_supporting_actor, X_test_best_supporting_actor, y_train_best_supporting_actor, y_test_best_supporting_actor)

,Train error,Validation error,Time in seconds
dummy,0.036,0.013,0.0177
decision tree,0.000,0.034,0.0196
kNN,0.017,0.008,0.0688
logistic regression,0.017,0.008,0.0262
random forest,0.000,0.008,0.1769
xgboost,0.005,0.008,0.1135
lgbm,0.000,0.008,0.0935


In [65]:
predict_supporting_actor_2019 = oscar_predict(X_train_best_supporting_actor, y_train_best_supporting_actor, best_supporting_actor_2019_identifiable, XGBClassifier())
predict_supporting_actor_2019

,Classifier,Probability of it Winning
A Beautiful Day in the Neighborhood,No,0.00345542
The Two Popes,No,0.00076545
The Irishman,No,0.0743856
Once Upon a Time... in Hollywood,No,0.223604


In [66]:
normalized_winner(predict_supporting_actor_2019)

,Chance of Winning
A Beautiful Day in the Neighborhood,1.14%
The Two Popes,0.25%
The Irishman,24.61%
Once Upon a Time... in Hollywood,73.99%


### Base Model Best Supporting Actress

In [67]:
diff_class_ml(X_train_best_supporting_actress, X_test_best_supporting_actress, y_train_best_supporting_actress, y_test_best_supporting_actress)

,Train error,Validation error,Time in seconds
dummy,0.039,0.008,0.0175
decision tree,0.000,0.025,0.0168
kNN,0.019,0.004,0.0629
logistic regression,0.017,0.004,0.0252
random forest,0.000,0.004,0.1704
xgboost,0.000,0.021,0.1071
lgbm,0.000,0.008,0.0806


In [74]:
predict_supporting_actress_2019 = oscar_predict(X_train_best_supporting_actress, y_train_best_supporting_actress, best_supporting_actress_2019_identifiable, LGBMClassifier()
                                               )
predict_supporting_actress_2019

,Classifier,Probability of it Winning
Richard Jewell,No,0.29851
Bombshell,No,0.136844
Jojo Rabbit,No,0.0389302
Little Woman,No,0.00713643
Marriage Story,No,0.0132644
Once Upon a Time... in Hollywood,No,0.414366


In [75]:
normalized_winner(predict_supporting_actress_2019)

,Chance of Winning
Richard Jewell,32.84%
Bombshell,15.05%
Jojo Rabbit,4.28%
Little Woman,0.79%
Marriage Story,1.46%
Once Upon a Time... in Hollywood,45.58%
